# Install Packages and Setup Variables


In [1]:
!pip install -q huggingface-hub==0.33.0 llama-index==0.12.43 llama-index-vector-stores-chroma==0.4.1 google-generativeai==0.8.5 \
                llama-index-llms-gemini==0.5.0 openai==1.92.0 chromadb==0.6.0 jedi==0.19.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 514.8/514.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.4/753.4 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.8 MB/s eta 0

In [2]:
import os

# Set the following API Keys in the Python environment. Will be used later.
# os.environ["OPENAI_API_KEY"] = "[OPENAI_API_KEY]"
# os.environ["GOOGLE_API_KEY"] = "<YOUR_API_KEY>"

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
os.environ["GOOGLE_API_KEY"] = userdata.get('Google_api_key')

# Load a Model


In [3]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(temperature=1, model="gpt-4o-mini")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

# Load Indexes


Downloading vector store from Huggingface hub

In [4]:
from huggingface_hub import hf_hub_download
vectorstore = hf_hub_download(repo_id="jaiganesan/ai_tutor_knowledge", filename="vectorstore.zip",repo_type="dataset",local_dir="/content")

vectorstore.zip:   0%|          | 0.00/97.2M [00:00<?, ?B/s]

In [5]:
!unzip /content/vectorstore.zip

Archive:  /content/vectorstore.zip
   creating: ai_tutor_knowledge/
   creating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/length.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/index_metadata.pickle  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/link_lists.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/header.bin  
  inflating: ai_tutor_knowledge/684af133-f877-4230-bde4-575cf53b6688/data_level0.bin  
  inflating: ai_tutor_knowledge/chroma.sqlite3  


In [6]:
# Load the vector store from the local storage.
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

db2 = chromadb.PersistentClient(path="/content/ai_tutor_knowledge")
chroma_collection = db2.get_or_create_collection("ai_tutor_knowledge")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [7]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding

# Create the index based on the vector store.
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [8]:
query_engine = index.as_query_engine(similarity_top_k=10)

res = query_engine.query("Explain how Advance RAG works?")

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


In [9]:
res.response

'The concept of Advance RAG primarily revolves around enhancing the fundamentals of Retrieval Augmented Generation (RAG) systems through novel methodologies. One notable approach is the SPECULATIVE RAG framework, which introduces a divide-and-conquer strategy to improve reasoning capabilities while maintaining processing speed. \n\nSPECULATIVE RAG employs a smaller specialist language model, referred to as the RAG drafter, to quickly generate multiple answer drafts from retrieved documents. This allows for rapid exploration of diverse perspectives on the provided information. Following this drafting phase, a larger generalist language model, known as the RAG verifier, evaluates these drafts, selecting the most appropriate one based on its rationale, and incorporates it into the final output.\n\nAdditionally, this framework addresses limitations found in traditional RAG models, such as latency issues and the need for efficient document processing. By partitioning retrieved documents int

# RankGPT


In [10]:
from llama_index.core.postprocessor.rankGPT_rerank import RankGPTRerank

rankGPT = RankGPTRerank(top_n=3,llm=Settings.llm, verbose=True)

In [11]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[rankGPT])

res = query_engine.query("Explain how Retrieval Augmented Generation (RAG) works?")

After Reranking, new rank list for nodes: [1, 0, 3, 9, 8, 7, 6, 2, 5, 4]

In [12]:
res.response

"Retrieval-Augmented Generation (RAG) works by combining two main components: retrieval and generation.\n\n1. **Retrieval Component**: This part is responsible for extracting relevant information from external knowledge sources. It involves:\n   - **Indexing**: Organizing documents using methods like inverted indexes for sparse retrieval or dense vector encoding for dense retrieval, making it easier to find relevant information.\n   - **Searching**: When a user query is input, the system utilizes the indexes to fetch pertinent documents. This process can be refined using rerankers to improve the order of the retrieved results based on relevance.\n\n2. **Generation Component**: This component uses the documents retrieved and the user's query to generate coherent responses. It works by:\n   - **Prompting**: Techniques such as Chain of Thought and Tree of Thought are employed to guide the generation process, enhancing the quality of the output.\n   - **Inferencing**: The model interprets 

In [13]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text.strip())
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 1c324686-fad7-4a41-bfd3-d44f9612ca91
Title	 Evaluation of Retrieval-Augmented Generation: A Survey:Research Paper Information
Text	 Authors: Hao Yu, Aoran Gan, Kai Zhang, Shiwei Tong, Qi Liu, and Zhaofeng Liu.Numerous studies of Retrieval-Augmented Generation (RAG) systems have emerged from various perspectives since the advent of Large Language Models (LLMs). The RAG system comprises two primary components: Retrieval and Generation. The retrieval component aims to extract relevant information from various external knowledge sources. It involves two main phases, indexing and searching. Indexing organizes documents to facilitate efficient retrieval, using either inverted indexes for sparse retrieval or dense vector encoding for dense retrieval. The searching component utilizes these indexes to fetch relevant documents based on the user's query, often incorporating the optional rerankers to refine the ranking of the retrieved documents. The generation component utilizes the retr

# Custom Postprocessor


## The `Judger` Function


The following function will query GPT-4o to retrieve the top three nodes that has highest similarity to the asked question.


In [14]:
from pydantic import BaseModel
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts import PromptTemplate


def judger(nodes, query):

    # The model's output template
    class OrderedNodes(BaseModel):
        """A node with the id and assigned score."""

        node_id: list
        score: list

    # Prepare the nodes and wrap them in <NODE></NODE> identifier, as well as the query
    the_nodes = ""
    for idx, item in enumerate(nodes):
        the_nodes += f"<NODE{idx+1}>\nNode ID: {item.node_id}\nText: {item.text}\n</NODE{idx+1}>\n"

    query = "<QUERY>\n{}\n</QUERY>".format(query)

    # Define the prompt template
    prompt_tmpl = PromptTemplate(
        """
    You receive a qurey along with a list of nodes' text and their ids. Your task is to assign score
    to each node based on its contextually closeness to the given query. The final output is each
    node id along with its proximity score.
    Here is the list of nodes:
    {nodes_list}

    And the following is the query:
    {user_query}

    Score each of the nodes based on their text and their relevancy to the provided query.
    The score must be a decimal number between 0 an 1 so we can rank them."""
    )

    # Define the an instance of GPT-4 and send the request
    llm = OpenAI(model="gpt-4o-mini")
    ordered_nodes = llm.structured_predict(
        OrderedNodes, prompt_tmpl, nodes_list=the_nodes, user_query=query
    )

    return ordered_nodes

## Define Postprocessor


The following class will use the `judger` function to rank the nodes, and filter them based on the ranks.


In [15]:
from typing import List, Optional
from llama_index.core import QueryBundle
from llama_index.core.postprocessor.types import BaseNodePostprocessor
from llama_index.core.schema import NodeWithScore


class OpenaiAsJudgePostprocessor(BaseNodePostprocessor):
    def _postprocess_nodes(
        self, nodes: List[NodeWithScore], query_bundle: Optional[QueryBundle]
    ) -> List[NodeWithScore]:

        r = judger(nodes, query_bundle)

        node_ids = r.node_id
        scores = r.score

        sorted_scores = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        num_nodes_to_select = min(3, len(sorted_scores))
        top_nodes = [sorted_scores[i][0] for i in range(num_nodes_to_select)]
        # top_nodes = [sorted_scores[i][0] for i in range(3)]

        selected_nodes_id = [node_ids[item] for item in top_nodes]

        final_nodes = []
        for item in nodes:
            if item.node_id in selected_nodes_id:
                final_nodes.append(item)

        return final_nodes

In [16]:
judge = OpenaiAsJudgePostprocessor()

## Query Engine with Postprocessor


In [20]:
# Define a query engine that is responsible for retrieving related pieces of text,
# and using a LLM to formulate the final answer.
# The `node_postprocessors` function will be applied to the retrieved nodes.
query_engine = index.as_query_engine(similarity_top_k=10, node_postprocessors=[judge])

res = query_engine.query("Compare Retrieval Augmented Generation (RAG) and Parameter efficient Finetuning (PEFT)")

In [21]:
res.response

'A comparison between Retrieval-Augmented Generation (RAG) and Parameter Efficient Fine-Tuning (PEFT) reveals key differences in their approaches to optimizing machine learning models. RAG combines retrieval mechanisms with generation, focusing on fine-tuning both the retriever and generator to enhance performance. This integration aims to effectively utilize context for generating content while ensuring relevant passages are retrieved.\n\nConversely, PEFT emphasizes improving model performance with fewer trainable parameters, thus reducing computational overhead. This approach seeks to maintain or improve performance without the complexity associated with training large models extensively.\n\nWhile RAG aims to integrate two components for optimal performance, PEFT streamlines the fine-tuning process by focusing on efficiency, often allowing for rapid adaptation to new tasks or domains with minimal resource usage. The choice between these strategies would depend on the specific goals a

In [22]:
# Show the retrieved nodes
for src in res.source_nodes:
    print("Node ID\t", src.node_id)
    print("Title\t", src.metadata["title"])
    print("Text\t", src.text)
    print("Score\t", src.score)
    print("-_" * 20)

Node ID	 8760ba85-ac25-46ee-813c-146a46fdf715
Title	 Searching for Best Practices in Retrieval-Augmented Generation:Retriever and Generator Fine-tuning
Text	 Fine-tuning within the RAG framework is crucial for optimizing both retrievers and generators. Some research focuses on fine-tuning the generator to better utilize retriever context [30–32], ensuring faithful and robust generated content. Others fine-tune the retriever to learn to retrieve beneficial passages for the generator [33–35]. Holistic approaches treat RAG as an integrated system; fine-tuning both retriever and generator together to enhance overall performance [36–38], despite increased complexity and integration challenges. Several surveys have extensively discussed current RAG systems, covering aspects like text generation [7, 8], integration with LLMs [6, 39], multimodal [40], and AI-generated content [41]. While these surveys provide comprehensive overviews of existing RAG methodologies, selecting the appropriate appr